### Exercise 1: Legal Sentences Analysis

We are given a series of legal sentences (in Spanish), and we would like to:

* Extract the companies that are mentioned in there.
* Extract the persons that are mentioned in there.

Which companies and persons have the most mentions in the dataset?

In [61]:
# you can start with the following code:

In [63]:
import pandas as pd

df = pd.read_excel('legal.xlsx')[:100]

In [64]:
#set max column width

pd.set_option('display.max_colwidth', -1)

df.sample(5)

/var/folders/l_/k13w4mhd5hv4bddxwqz8qdfw0000gn/T/ipykernel_7822/2196259785.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,nref,fallo
40,2003/203935,"I.- QUE DEBEMOS DECLARAR Y DECLARAMOS LA INADMISIBILIDAD DEL RECURSO CONTENCIOSO-ADMINISTRATIVO INTERPUESTO POR EL PROCURADOR D. GERMÁN APALATEGUI CARASA EN NOMBRE Y REPRESENTACIÓN DE DOÑA Mercedes CONTRA CONVENIO URBANÍSTICO SUSCRITO POR EL AYUNTAMIENTO DE DEBA Y Dª Antonia, APROBADO POR EL AYUNTAMIENTO EL 25 DE MARZO DE 1993 Y SUSCRITO EL 30 SIGUIENTE, POR EL QUE SE ACUERDA LA CESIÓN DE UNA PARCELA DE 316,60 m2 A CAMBIO DE UN APROVECHAMIENTO URBANÍSTICO DE 2.849 m3 A SITUAR EN LA PARTE DE SU PROPIEDAD A CONCRETAR MEDIANTE UN ESTUDIO DE DETALLE. SIN IMPOSICIÓN DE COSTAS. II.- QUE ESTIMANDO EL RECURSO INTERPUESTO POR EL PROCURADOR D. GERMAN APALATEGUI CARASA EN NOMBRE Y REPRESENTACIÓN DE DOÑA Mercedes CONTRA LA DESESTIMACIÓN PRESUNTA DEL RECURSO DE REPOSICIÓN INTERPUESTO CONTRA EL ACUERDO DE 25 DE ENERO DE 2001 DEL AYUNTAMIENTO DE DEBA POR EL QUE SE APRUEBA DEFINITIVAMENTE EL ESTUDIO DE DETALLE RELATIVO A LA ZONA 9 DE LAS NORMAS SUBSIDIARIAS DE PLANEAMIENTO MUNICIPAL DE DEBA, ASÍ COMO EL ACUERDO DEL CITADO AYUNTAMIENTO DE 24 DE ABRIL DE 2001 POR EL QUE SE DESESTIMA DE FORMA EXPRESA EL RECURSO DE REPOSICIÓN INTERPUESTO, Y; DEBEMOS: PRIMERO.- DECLARAR LA DISCONFORMIDAD A DERECHO DE LOS ACTOS RECURRIDOS QUE CONSECUENTEMENTE ANULAMOS. SEGUNDO.- SIN IMPOSICIÓN DE LAS COSTAS. Notifíquese esta resolución a las partes, advirtiéndoles que contra la misma cabe interponer RECURSO DE CASACIÓN ante la Sala de lo Contencioso - administrativo del Tribunal Supremo, el cual, en su caso, se preparará ante esta Sala en el plazo de DIEZ DÍAS, contados desde el siguiente al de la notificación de esta resolución, mediante escrito en el que deberá manifestarse la intención de interponer el recurso, con sucinta exposición de la concurrencia de los requisitos exigidos. Así por esta nuestra Sentencia, de la que se llevará testimonio a los autos, lo pronunciamos, mandamos y firmamos. Publicación. Leída y publicada fue la anterior sentencia por el Ilustrísimo Señor Magistrado Ponente de la misma, estando celebrando audiencia pública la Sala de lo Contencioso Administrativo del Tribunal Superior de Justicia del País Vasco, en el día de su fecha, de lo que yo el/la Secretario doy fe en BILBAO a 7 de noviembre de 2003."
20,2017/213715,"En virtud de lo expuesto, LA SALA ACUERDA : Inadmitir el recurso de queja interpuesto por representación procesal de D.ª Lorenza y D.ª Marí Juana contra la providencia de fecha 26 de mayo de 2017, de la Audiencia Provincial de Toledo (Sección 2.ª), en el rollo 291/2014, debiendo ponerse esta resolución en conocimiento de la referida Audiencia para que conste en los autos. La parte recurrente perderá el depósito constituido. Contra la presente resolución no cabe recurso alguno. Así lo acuerdan, mandan y firman los Excmos. Sres. Magistrados indicados al margen, de lo que como Secretario, certifico."
48,2004/46741,"Que estimamos parcialmente el recurso contencioso-administrativo interpuesto por la Procuradora Dª AMAYA MARÍA GONZÁLEZ CELAYA, en representación de la entidad ""Recreativos Cedeira, S.L."", contra acuerdo del TEAR de Galicia, adoptado en sesión de fecha 27 de septiembre de 2001, que estima en parte reclamación económico-administrativa formulada contra resolución de fecha 24 de mayo de 1999, de la Jefa de Servicio de Gestión Tributaria de la Delegación en A Coruña de la Consellería de Economía e Facenda de la Xunta de Galicia, por la que se estima parcialmente solicitud de devolución de ingresos indebidos correspondientes a la Tasa Fiscal sobre el Juego, ejercicios 1992 a 1996, y anulamos parcialmente dicho acuerdo, por contrario a Derecho, en el particular relativo a la denegación de la devolución de lo ingresado en el primer y segundo trimestres del año 1992 y en el primer trimestre del año 1995 en concepto de incremento anual de la cuota fija de la Tasa Fiscal sobre el Juego relativa a máquinas recreativas tipo ""B"", debiendo procederse a la devolución de lo ingresado por tal concepto y

Load a suitable variant of the GLiNER model

In [65]:
from gliner import GLiNER

# Initialize GLiNER with the base model
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

/Users/victorgallego/miniforge3/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


What are going to be the labels to extract?

In [66]:
labels = ["Company", "Person"]

Process all the sentences with the model and display the results.

In [67]:
texts = df['fallo'].tolist()

results = []

for text in texts:
    result = model.predict_entities(text, labels, threshold=0.5)
    results.append(result)

In [68]:
# flatten the list
results = [item for sublist in results for item in sublist]

df_entities = pd.DataFrame(results)

In [69]:
df_entities

,start,end,text,label,score
0,121,136,Nicanor Modesto,Person,0.929545
1,74,90,María Inmaculada,Person,0.585221
2,328,337,D. Manuel,Person,0.925554
3,1420,1441,Doña María Inmaculada,Person,0.848120
4,1781,1790,D. Manuel,Person,0.897005
...,...,...,...,...,...
513,466,488,Carles Viver Pi-Sunyer,Person,0.860278
514,491,511,Enrique Ruiz Vadillo,Person,0.895998
515,514,547,Manuel Jiménez de Parga y Cabrera,Person,0.916679
516,550,570,Tomás S. Vives Antón,Person,0.883622


Which companies and persons are the most mentioned in the data?

In [72]:
df_entities[df_entities.label == "Company"]['text'].value_counts()

BANCO SANTANDER S.A.                                                 3
Telefónica de España S.A.                                            2
Fagor                                                                2
TODOLUZ MALAGA SL                                                    2
TODOLUZ MADRID SL                                                    2
GLOBAL MANUFACTURING & TRADING COMPANY SL                            2
M., S.A.                                                             2
C., S.A.                                                             2
HIPERDELUZ SL                                                        2
H., S.A.                                                             2
Banco Santander S.A.                                                 2
Constructora T., S.A.                                                2
Seguros B.                                                           2
ILUNION CEE OUTSOURCING SAU                                          1
Cámara

In [73]:
df_entities[df_entities.label == "Person"]['text'].value_counts()

D. Manuel                   4
D. José                     4
D. Agapito y Dª Adolfina    4
D. Andrés                   3
Santos                      3
                           ..
Dª Camila                   1
D. Pablo                    1
Dª María                    1
D. Arsenio                  1
Pablo García Manzano        1
Name: text, Length: 369, dtype: int64

### Exercise 2: Customer Review Analysis

Now, we will analyze the reviews dataset from Disneyland theme parks.

We would like to extract the attractions that are mentioned in the reviews, with their associated sentiment.

As a starter, you can use the following code:

In [28]:
df = pd.read_csv('DisneylandReviews.csv', sep=',', encoding='iso-8859-1')[:100]

In [32]:
texts = df['Review_Text'].tolist()

1. Use a suitable model to perform sentiment analysis and classify each review.

Create a sentiments list that is the same length as the reviews list, with the sentiment of each review.

In [42]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis", truncation=True)

sentiments = []

for text in texts:
    result = pipe(text)[0]['label']
    sentiments.append(result)

2. Define the label to extract the attractions of the reviews text:

In [43]:
labels = ["Attraction"]

3. Apply the GLiNER model to the reviews and extract the attractions mentioned in the reviews.
   
Also add the information of the corresponding sentiment to each entity extracted.

In [46]:
results = []

for sentiment, text in zip(sentiments, texts):
    result = model.predict_entities(text, labels, threshold=0.5)
    for entity in result:
        entity['sentiment'] = sentiment
    results.append(result)

In [47]:
result

[{'start': 51,
  'end': 67,
  'text': 'Disneyland Paris',
  'label': 'Attraction',
  'score': 0.6424593925476074,
  'sentiment': 'POSITIVE'},
 {'start': 287,
  'end': 301,
  'text': 'Space Mountain',
  'label': 'Attraction',
  'score': 0.9413168430328369,
  'sentiment': 'POSITIVE'}]

4. Convert it to a dataframe and display the results.

In [48]:
# flatten the list
results = [item for sublist in results for item in sublist]

df_entities = pd.DataFrame(results)

5. Compute the average sentiment between all the extracted attractions
(Assume positive is 1 and negative is 0).

In [50]:
df_entities['sentiment'].replace({'POSITIVE': 1, 'NEGATIVE': 0}, inplace=True)

In [52]:
df_entities['sentiment'].mean()

0.7291666666666666

6. Compute the average sentiment, grouping by the attraction 

In [56]:
df_entities.groupby('text')['sentiment'].mean().reset_index().sort_values('sentiment', ascending=False)

,text,sentiment
0,Adventure Land,1.0
41,Mickey s Wondrous Book,1.0
69,Tomorrow land,1.0
68,The park,1.0
67,The lion king show,1.0
...,...,...
77,Winnie the Pooh,0.0
76,Walt Disney World,0.0
21,Explorers lodge,0.0
74,Toy story land,0.0


7. Repeat a similar analysis, but extracting Food.

In [57]:
labels = ["Food"]

results = []

for sentiment, text in zip(sentiments, texts):
    result = model.predict_entities(text, labels, threshold=0.5)
    for entity in result:
        entity['sentiment'] = sentiment
    results.append(result)

In [58]:
# flatten the list
results = [item for sublist in results for item in sublist]

df_entities = pd.DataFrame(results)

df_entities['sentiment'].replace({'POSITIVE': 1, 'NEGATIVE': 0}, inplace=True)

In [60]:
df_entities['sentiment'].mean()

0.7073170731707317

In [59]:
df_entities.groupby('text')['sentiment'].mean().reset_index().sort_values('sentiment', ascending=False)

,text,sentiment
16,caramel popcorn,1.000000
12,Vegetarian food,1.000000
29,snacks,1.000000
26,hire pram,1.000000
25,halal foods,1.000000
24,halal food,1.000000
23,food options,1.000000
19,drinks,1.000000
18,disneyland,1.000000
15,brilliant food,1.000000
